In [1]:
import requests
import requests_mock
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException

# Creating L2VPN

In [2]:
# Define a mock response dictionaries for each of the required scenarios

# Successful object creation
mock_response_successful = {
    "service_id": "12345",
}

# Error Codes for SDXExceptions
mock_response_unsuccessful_400 = {
    "400": "Request does not have a valid JSON or body is incomplete/incorrect"
}
mock_response_unsuccessful_401 = {
    "401": "Not Authorized"
}
mock_response_unsuccessful_402 = {
    "402": "Request not compatible (e.g., P2MP L2VPN requested, but only P2P supported)"
}
mock_response_unsuccessful_409 = {
    "409": "L2VPN Service already exists"
}
mock_response_unsuccessful_410 = {
    "410": "Can't fulfill the strict QoS requirements"
}
mock_response_unsuccessful_411 = {
    "411": "Scheduling not possible"
}
mock_response_unsuccessful_422 = {
    "422": "Attribute not supported by the SDX-LC/OXPO"
}

In [3]:
# Define the required attributes
url = "http://example.com"
client_name = "Test L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:sax.br:Rtr01:50", "vlan": "any"},
    {"port_id": "urn:sdx:port:ampath.net:Ampath3:50", "vlan": "any"}
]

In [4]:
# Create the L2VPN object
client = SDXClient(url, client_name, client_endpoints)

In [5]:
client

SDXClient(base_url=http://example.com, name=Test L2VPN, endpoints=[{'port_id': 'urn:sdx:port:sax.br:Rtr01:50', 'vlan': 'any'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': 'any'}])

In [6]:
# Create a mocker for requests for the unsuccessful case
with requests_mock.Mocker() as mock_requests:
    # Mock unsuccessful L2VPN creation
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0",
        json=mock_response_unsuccessful_422,
        status_code=422  # Set status code for unsuccessful creation
    )
    
    # Debug print to verify the mock setup
    print(f"Mock for request exception set with URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
        print("Request successful.")
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  

Mock for request exception set with URL: http://example.com/l2vpn/1.0
L2VPN creation request sent to http://example.com/l2vpn/1.0. It may take several seconds to receive a response.
Mocked server response:
 {"422": "Attribute not supported by the SDX-LC/OXPO"}
L2VPN creation failed: Attribute not supported by the SDX-LC/OXPO


In [7]:
# Create a mocker for requests to simulate a RequestException
with requests_mock.Mocker() as mock_requests:
    # Mock a request exception scenario (e.g., connection error)
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0",
        exc=requests.exceptions.RequestException("Connection error")
    )
    
    # Debug print to verify the mock setup
    print(f"Mock for request exception set with URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  
    except requests.exceptions.RequestException as e:
        print(f"Request exception occurred: {e}")  

An error occurred while creating L2VPN: Connection error


Mock for request exception set with URL: http://example.com/l2vpn/1.0
L2VPN creation request sent to http://example.com/l2vpn/1.0. It may take several seconds to receive a response.
An error occurred while creating L2VPN: Connection error
L2VPN creation failed: An error occurred while creating L2VPN: Connection error


In [8]:
# Create a mocker for requests
with requests_mock.Mocker() as mock_requests:
    # Mock successful L2VPN creation
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0", 
        json=mock_response_successful,
        status_code=201  # Set status code for successful creation
    )
    
    # Adding debug statement to confirm URL and method
    print(f"Mocking URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
        print("L2VPN creation successful!")
        print(f"Response: {response}")  
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  

Mocking URL: http://example.com/l2vpn/1.0
L2VPN creation request sent to http://example.com/l2vpn/1.0. It may take several seconds to receive a response.
L2VPN creation successful!
Response: {'service_id': '12345'}


# Retrieve L2VPNs